## Import Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')
import datetime
import itertools
from collections import defaultdict

import math
import numpy as np
import pandas as pd
from scipy.stats import shapiro
import scipy.stats as stats
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from statsmodels.graphics.tsaplots import plot_pacf

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, KFold, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OrdinalEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, make_scorer
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.compose import make_column_selector as selector

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV, ElasticNet
from xgboost import XGBRegressor
import catboost as cb
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## Configs

In [3]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

## Load Data

In [4]:
dir_path = '/kaggle/input/store-sales-time-series-forecasting/'

oil_df = pd.read_csv(dir_path + 'oil.csv')
holidays_df = pd.read_csv(dir_path + 'holidays_events.csv')
stores_df = pd.read_csv(dir_path + 'stores.csv')
train_df = pd.read_csv(dir_path + 'train.csv')
test_df = pd.read_csv(dir_path + 'test.csv')
trnsctns_df = pd.read_csv(dir_path + 'transactions.csv')
submissions_df = pd.read_csv(dir_path + 'sample_submission.csv')

In [5]:
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [6]:
holidays_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [7]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [8]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0
1,1,2013-01-01,1,BABY CARE,0.00,0
2,2,2013-01-01,1,BEAUTY,0.00,0
3,3,2013-01-01,1,BEVERAGES,0.00,0
4,4,2013-01-01,1,BOOKS,0.00,0


In [9]:
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [10]:
trnsctns_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [11]:
submissions_df.head()

,id,sales
0,3000888,0.00
1,3000889,0.00
2,3000890,0.00
3,3000891,0.00
4,3000892,0.00


## Data Pre-processing

In [12]:
# transform sales to log scale
train_df['sales'] = np.log1p(train_df['sales'])

#### Change 'date' column type

In [13]:
oil_df['date'] = pd.to_datetime(oil_df['date'], format = "%Y-%m-%d")
holidays_df['date'] = pd.to_datetime(holidays_df['date'], format = "%Y-%m-%d")
trnsctns_df['date'] = pd.to_datetime(trnsctns_df['date'], format = "%Y-%m-%d")
train_df['date'] = pd.to_datetime(train_df['date'], format = "%Y-%m-%d")
test_df['date'] = pd.to_datetime(test_df['date'], format = "%Y-%m-%d")

#### Fill missing values in datasets

In [14]:
train_data_strt_dt = train_df['date'].min()
train_data_end_dt = train_df['date'].max()
train_dt_rnge = pd.date_range(start=train_data_strt_dt, end=train_data_end_dt)
train_missing_dts = train_dt_rnge.difference(train_df['date'])

test_data_strt_dt = test_df['date'].min()
test_data_end_dt = test_df['date'].max()
test_dt_rnge = pd.date_range(start=test_data_strt_dt, end=test_data_end_dt)
test_missing_dts = test_dt_rnge.difference(test_df['date'])

print(f"Missing dates in training set: {train_missing_dts}")
print(f"Missing dates in test set: {test_missing_dts}")

Missing dates in training set: DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)
Missing dates in test set: DatetimeIndex([], dtype='datetime64[ns]', freq=None)


Create a Multi-index variable

In [15]:
multi_index = pd.MultiIndex.from_product([pd.date_range(train_data_strt_dt, train_data_end_dt),
                                          train_df.store_nbr.unique(),
                                          train_df.family.unique()],
                                         names=['date','store_nbr','family'],)
train_df = train_df.set_index(['date','store_nbr','family']).reindex(multi_index).reset_index()

train_df.head()

,date,store_nbr,family,id,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,0.00
1,2013-01-01,1,BABY CARE,1.00,0.00,0.00
2,2013-01-01,1,BEAUTY,2.00,0.00,0.00
3,2013-01-01,1,BEVERAGES,3.00,0.00,0.00
4,2013-01-01,1,BOOKS,4.00,0.00,0.00


Fill missing values with 0s

In [16]:
train_df[['sales','onpromotion']] = train_df[['sales','onpromotion']].fillna(0.)

# Apply linear interpolation to the "id" column to estimate the missing values based on the linear relationship between adjacent data points. 
train_df.id = train_df.id.interpolate(method="linear")

Add an additional column in both training and test sets to separate those two

In [17]:
#train_df['test'] = 0
#test_df['test'] = 1

#### Oil Data - Re-index by adding missing dates

In [18]:
# Create a date range from the start of training data to the end of test data
date_range = pd.date_range(train_data_strt_dt, test_data_end_dt)

# Create a DataFrame with the date range
date_df = pd.DataFrame({'date': date_range})

# Merge the date_df with oil_df using an outer join
oil_df = oil_df.merge(date_df, on = 'date', how = 'outer')

# Sort the DataFrame by date and reset the index
oil_df = oil_df.sort_values('date', ignore_index=True)

# fill missing values using linear interpolation
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].interpolate(method="linear", limit_direction="both")

oil_df.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,93.15


#### Transactions Data - Fill in the missing values for transactions using interpolation, except for days with zero sales

In [19]:
# Calculate the number of unique store numbers
num_store = train_df['store_nbr'].nunique()

# Calculate the number of days in the training period
train_len = (train_data_end_dt - train_data_strt_dt).days + 1

# Calculate the number of records where sales are zero
num_zero_sales = (train_df.groupby(["date", "store_nbr"])['sales'].sum() == 0).sum()

# Calculate the total number of expected records
total_rec = num_store * train_len

# Calculate the current number of records
curr_rec = len(trnsctns_df.index)

# Calculate the number of missing records
missing_rec = total_rec - curr_rec - num_zero_sales

# Total sales for each store
store_sales = train_df.groupby(["date", "store_nbr"]).sales.sum().reset_index()

# Re-index transaction data
trnsctns_df = trnsctns_df.merge(store_sales, on=["date", "store_nbr"],how="outer").sort_values(["date", "store_nbr"],ignore_index=True)


# Fill missing values with 0s for days with zero sales
trnsctns_df.loc[trnsctns_df.sales.eq(0), "transactions"] = 0

# Drop the "sales" column
trnsctns_df = trnsctns_df.drop(columns=["sales"])

# Fill remaining missing values using linear interpolation within each "store_nbr" group
trnsctns_df["transactions"] = trnsctns_df.groupby("store_nbr")["transactions"].transform(lambda x: x.interpolate(method="linear", limit_direction="both"))

## Feature Engineering

Function for adding lag features

In [20]:
def lag_features(df, lags):
    for lag in lags:
        df[f"sales_t-{lag}"] = df.groupby(["store_nbr", "family"])["sales"].transform(lambda x: x.shift(lag))
    return df

Function for rolling average features

In [21]:
# Random Noise
def random_noise(df):
    return np.random.normal(scale=2.0, size=(len(df),))

In [22]:
def roll_mean_features(df, windows):
    for window in windows:
        df['sales_roll_mean_' + str(window)] = df.groupby(["store_nbr", "family"])['sales']. \
                                                          transform(
            lambda x: x.shift(16).rolling(window=window, min_periods=7, win_type="triang").mean()) + random_noise(df)
    return df

Exponential Weighted Moving Average (EWM) Features

In [23]:
def ewm_features(df, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            df['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                df.groupby(["store_nbr", "family"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return df

Add new columns that provide temporal information about the data

In [24]:
def create_date_features(df):
    df['month'] = df.date.dt.month
    df['day_of_month'] = df.date.dt.day
    df['day_of_year'] = df.date.dt.dayofyear
    #df['week_of_year'] = df.date.dt.weekofyear
    df['day_of_week'] = df.date.dt.dayofweek
    df['year'] = df.date.dt.year
    return df

Function to fill missing values

In [25]:
def fill_na(df):
    df['holiday_type'] = df['holiday_type'].fillna('Common')
    df['locale'] = df['locale'].fillna('Common')
    df['description'] = df['description'].fillna('Unknown')
    df['transferred'] = df['transferred'].fillna(False)
    df['dcoilwtico'] = df['dcoilwtico'].fillna(method='backfill')
    return df

Master dataset creation

In [26]:
def merge_data(df):
    df = df \
        .merge(stores_df, left_on="store_nbr", right_on="store_nbr", how="left") \
        .rename(columns={"type": "store_type"}) \
        .merge(trnsctns_df, left_on=["date", "store_nbr"], right_on=["date", "store_nbr"], how="left") \
        .merge(holidays_df, left_on="date", right_on="date", how="left") \
        .drop_duplicates(subset="id") \
        .rename(columns={"type": "holiday_type"}) \
        .merge(oil_df, left_on="date", right_on="date", how="left") 
    return df

In [27]:
use_cols = ['date','store_nbr','family','sales','onpromotion','cluster','holiday_type', 
            'locale','description','transferred','dcoilwtico']

categorical_features = ['family','holiday_type', 'locale','description']

In [28]:
data = pd.concat([train_df, test_df], axis=0)
data = merge_data(data)
data = data[use_cols]
data = fill_na(data)
data = create_date_features(data)
data = lag_features(data, lags = [*range(1, 16), 16,17,18,19,20,21,22,30,31,90,180,364])
data = roll_mean_features(data,[16,17,18,30])
data = ewm_features(data, [0.95, 0.9, 0.8, 0.5],[1, 7,30])

data.head()

,date,store_nbr,family,sales,onpromotion,cluster,holiday_type,locale,description,transferred,dcoilwtico,month,day_of_month,day_of_year,day_of_week,year,sales_t-1,sales_t-2,sales_t-3,sales_t-4,sales_t-5,sales_t-6,sales_t-7,sales_t-8,sales_t-9,sales_t-10,sales_t-11,sales_t-12,sales_t-13,sales_t-14,sales_t-15,sales_t-16,sales_t-17,sales_t-18,sales_t-19,sales_t-20,sales_t-21,sales_t-22,sales_t-30,sales_t-31,sales_t-90,sales_t-180,sales_t-364,sales_roll_mean_16,sales_roll_mean_17,sales_roll_mean_18,sales_roll_mean_30,sales_ewm_alpha_095_lag_1,sales_ewm_alpha_095_lag_7,sales_ewm_alpha_095_lag_30,sales_ewm_alpha_09_lag_1,sales_ewm_alpha_09_lag_7,sales_ewm_alpha_09_lag_30,sales_ewm_alpha_08_lag_1,sales_ewm_alpha_08_lag_7,sales_ewm_alpha_08_lag_30,sales_ewm_alpha_05_lag_1,sales_ewm_alpha_05_lag_7,sales_ewm_alpha_05_lag_30
0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01,1,BABY CARE,0.00,0.00,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01,1,BEAUTY,0.00,0.00,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,1,BEVERAGES,0.00,0.00,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01,1,BOOKS,0.00,0.00,13,Holiday,National,Primer dia del ano,False,93.14,1,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
data[categorical_features] = data[categorical_features].astype('category')

## Split the data

In [30]:
data = data.query("date > '2013-12-31'")

X = data.query("date <= '2017-08-15'")
X.drop(['date'], inplace=True, axis=1)
Y = X['sales']
X = X.drop(['sales', 'year'], axis=1)

X_test = data.query("date > '2017-08-15'")
X_test.drop(['date', 'year'], inplace=True, axis=1)
X_test = X_test.drop(['sales'], axis=1)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"X_train: {X_train.shape}")
print(f"Y_train: {Y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"Y_val: {Y_val.shape}")
#print(f"X_test: {X_test.shape}")
#print(f"Y_test: {Y_test.shape}")

X_train: (1886068, 56)
Y_train: (1886068,)
X_val: (471518, 56)
Y_val: (471518,)


## Model Development

In [31]:
final_features = X_train.columns.to_list()

### Light GBM Regressor

In [33]:
lgb_params = {
    'metric': 'mse',
    'boosting_type' : 'gbdt',
    'num_leaves': 8,
    'learning_rate': 0.2,
    'max_depth': 7,
    'verbose': 0,
    'num_boost_round': 5000,
    'early_stopping_rounds': 200,
    'nthread': -1,
    'force_col_wise': True,
    'device_type': 'cpu',
}

train_dataset = lgb.Dataset(data=X_train, label=Y_train,
                            feature_name=final_features, categorical_feature=categorical_features)
val_dataset = lgb.Dataset(data=X_val, label=Y_val, reference=train_dataset,
                          feature_name=final_features, categorical_feature=categorical_features)

model = lgb.train(
    params=lgb_params, 
    train_set=train_dataset,
    valid_sets=[train_dataset, val_dataset],
    verbose_eval=100,
)

Training until validation scores don't improve for 200 rounds
[100]	training's l2: 0.174004	valid_1's l2: 0.176487
[200]	training's l2: 0.160962	valid_1's l2: 0.163902
[300]	training's l2: 0.154843	valid_1's l2: 0.158095
[400]	training's l2: 0.150881	valid_1's l2: 0.154291
[500]	training's l2: 0.147956	valid_1's l2: 0.151563
[600]	training's l2: 0.145859	valid_1's l2: 0.149656
[700]	training's l2: 0.143959	valid_1's l2: 0.148018
[800]	training's l2: 0.142164	valid_1's l2: 0.146419
[900]	training's l2: 0.140787	valid_1's l2: 0.145207
[1000]	training's l2: 0.139582	valid_1's l2: 0.144325
[1100]	training's l2: 0.138652	valid_1's l2: 0.143593
[1200]	training's l2: 0.137607	valid_1's l2: 0.142742
[1300]	training's l2: 0.136726	valid_1's l2: 0.142066
[1400]	training's l2: 0.135849	valid_1's l2: 0.1414
[1500]	training's l2: 0.135096	valid_1's l2: 0.14087
[1600]	training's l2: 0.134429	valid_1's l2: 0.140473
[1700]	training's l2: 0.133833	valid_1's l2: 0.140071
[1800]	training's l2: 0.13318	va

## Model Evaluation

In [34]:
def recursive_predict(model):
    global X_test
    output = np.array([])
    for day in range(16, 32):
        pred = model.predict(X_test.query(f"day_of_month == {day}"))
        pred[pred < 0] = 0
        output = np.concatenate([output, pred], axis=0)
        for k in range(day+1, 32):
            X_test.loc[X_test[X_test["day_of_month"] == k].index, f"sales_t-{k-day}"] = pred
    return output

In [40]:
# Metrics used to evaluate models
def metrics_regression(y_true, y_pred, dataset='Validation'):
    # MSE
    mse = mean_squared_error(y_true, y_pred) #!
    
    # RMSE (Root Mean Square Error)
    rmse = math.sqrt(mse)
    
    # R^2
    r2 = r2_score(y_true, y_pred)
    
    # MAE(mean absolute error)
    mae = mean_absolute_error(y_true, y_pred) #!
    
    # MAPE(mean absolute percentage error)
    mape = mean_absolute_percentage_error(y_true, y_pred) #!
    
    #SMAPE (symmetric mean absolute percentage error)
    smape = 1/len(y_true) * np.sum(2 * np.abs(y_pred-y_true) / (np.abs(y_true) + np.abs(y_pred))*100)
    
    header = f"{dataset} Set Performance"
    print("="*len(header), header, "="*len(header))
    print(f"MSLE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.4f}")
    print(f"SMAPE: {smape:.4f}")

In [41]:
Y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)
Y_val_pred[Y_val_pred < 0] = 0

metrics_regression(Y_val, Y_val_pred, dataset='Validation')

========================== Validation Set Performance ==========================
MSLE: 0.1350
RMSE: 0.3675
R2: 0.9811
MAE: 0.2259
MAPE: 165825214758999.6250
SMAPE: 44.0713


In [42]:
rec_pred_lgb = recursive_predict(model)

## Submission

In [43]:
final_submission = pd.DataFrame({'id': test_df['id'], 'sales': np.expm1(rec_pred_lgb)})
final_submission.to_csv('submission.csv', index=False)